<a href="https://colab.research.google.com/github/wafe29/Collab_TF_Connectivity/blob/main/TF_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon@NERV (connectivity)

Template of a scenario by Wafa compatible with timeflux : sc2_extract_one.yaml


In [ ]:

!pip install timeflux_dsp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.1/693.1 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 24.10.0 requires networkx>=3.0, but you have networkx 2.5.1 which is incompatible.
scikit-image 0.24.0

test

hello

In [26]:
import subprocess
#subprocess.run(["timeflux", "-d", str('/content/sc2_extract_one.yaml')])
output = subprocess.check_output(["timeflux", "-d", str('/content/sc2_extract_one.yaml')])
print(output.decode())


2024-12-03 16:15:30,498 INFO       timeflux     26390    MainProcess      Timeflux 0.17.1
2024-12-03 16:15:30,533 DEBUG      manager      26390    MainProcess      Worker spawned with PID 26398
Extracting EDF parameters from /content/Test-[2022.10.10-15.55.20].edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 253499  =      0.000 ...   506.998 secs...
the current channel names ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'FCz', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'Iz']
Time taken to load data: 1.12 seconds
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_End_Of_Session', 'OVTK_GDF_End_Of_Trial

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy.signal import welch
from scipy.fft import fftfreq, rfftfreq, fft, rfft
from scipy import signal
import matplotlib.pyplot as plt
from timeflux.core.node import Node

In [ ]:
class Coherence_Welch(Node):


    def __init__(self, rate=None, closed="right", **kwargs):
        """
        Args:
            rate (float|None): Nominal sampling rate of the input data. If `None`, the rate will be taken from the input meta/
            closed (str): Make the index closed on the `right`, `left` or `center`.
            kwargs:  Keyword arguments to pass to scipy.signal.welch function.
                            You can specify: window, nperseg, noverlap, nfft, detrend, return_onesided and scaling.
        """

        self._rate = rate
        self._closed = closed
        self._kwargs = kwargs
        self._set_default()

    def _set_default(self):
        # We set the default params if they are not specifies in kwargs in order to check that they are valid, in respect of the length and sampling of the input data.
        if "nperseg" not in self._kwargs.keys():
            self._kwargs["nperseg"] = 256
            self.logger.debug("nperseg := 256")
        if "nfft" not in self._kwargs.keys():
            self._kwargs["nfft"] = self._kwargs["nperseg"]
            self.logger.debug(
                "nfft := nperseg := {nperseg}".format(nperseg=self._kwargs["nperseg"])
            )
        if "noverlap" not in self._kwargs.keys():
            self._kwargs["noverlap"] = self._kwargs["nperseg"] // 2
            self.logger.debug(
                "noverlap := nperseg/2 := {noverlap}".format(
                    noverlap=self._kwargs["noverlap"]
                )
            )

    def _check_nfft(self):
        # Check validity of nfft at first chun
        if not all(
            i <= len(self.i.data)
            for i in [self._kwargs[k] for k in ["nfft", "nperseg", "noverlap"]]
        ):
            raise ValueError(
                "nfft, noverlap and nperseg must be greater than or equal to length of chunk."
            )
        else:
            self._kwargs.update(
                {
                    keyword: int(self._kwargs[keyword])
                    for keyword in ["nfft", "nperseg", "noverlap"]
                }
            )

    def update(self):
        # copy the meta
        self.o = self.i

        # When we have not received data, there is nothing to do
        if not self.i.ready():
            return

        # Check rate
        if self._rate:
            rate = self._rate
        elif "rate" in self.i.meta:
            rate = self.i.meta["rate"]
        else:
            raise ValueError(
                "The rate was neither explicitely defined nor found in the stream meta."
            )

        # At this point, we are sure that we have some data to process
        # apply welch on the data:
        self._check_nfft()

        trials_MI = []
        for tr in range(self.i.data.shape[0]):
          elec1_MI = []
          for elec1 in range(self.i.data.shape[2]):
            elec2_MI = []
            for elec2 in range(self.i.data.shape[2]):
              f, Cxy_MI = signal.coherence(self.i.data[tr,:,elec1], self.i.data[tr,:,elec2], fs=rate, window='hann',  **self._kwargs)
              elec2_MI.append(Cxy_MI)
            elec1_MI.append(elec2_MI)
          trials_MI.append(elec1_MI)

        NS = np.array(trials_MI).mean(1)
        NS.transpose(0,2,1)
        if self._closed == "left":
            time = self.i.data.index[-1]
        elif self._closed == "center":

            def middle(a):
                return int(np.ceil(len(a) / 2)) - 1

            time = self.i.data.index[middle(self.i.data)]
        else:  # right
            time = self.i.data.index[-1]
        # f is the frequency axis and Pxx the average power of shape (Nfreqs x Nchanels)
        # we reshape Pxx to fit the ('time' x 'freq' x 'space') dimensions
        self.o.data = xr.DataArray(
            np.stack([NS], 0),
            coords=[[time], f, self.i.data.columns],
            dims=["time", "frequency", "space"],
        )